# Data
application_train/application_test: data pelatihan dan pengujian utama dengan informasi tentang setiap aplikasi pinjaman di Home Credit. Setiap pinjaman memiliki barisnya masing-masing dan ditandai dengan fitur SK_ID_CURR. Data aplikasi pelatihan dilengkapi dengan TARGET yang menunjukkan 0: pinjaman telah dilunasi atau 1: pinjaman tidak dilunasi.

bureau: data mengenai kredit klien sebelumnya dari lembaga keuangan lain. Setiap kredit sebelumnya memiliki barisnya sendiri di biro, tetapi satu pinjaman dalam data aplikasi dapat memiliki beberapa kredit sebelumnya.

bureau_balance: data bulanan tentang kredit sebelumnya di biro. Setiap baris adalah satu bulan dari kredit sebelumnya, dan satu kredit sebelumnya dapat memiliki beberapa baris, satu untuk setiap bulan dari panjang kredit.

previous_application: aplikasi sebelumnya untuk pinjaman di Home Credit klien yang memiliki pinjaman dalam data aplikasi. Setiap pinjaman saat ini dalam data aplikasi dapat memiliki beberapa pinjaman sebelumnya. Setiap aplikasi sebelumnya memiliki satu baris dan diidentifikasi oleh fitur SK_ID_PREV.

POS_CASH_BALANCE: data bulanan tentang titik penjualan sebelumnya atau pinjaman tunai yang dimiliki klien dengan Home Credit. Setiap baris adalah satu bulan dari titik penjualan atau pinjaman tunai sebelumnya, dan satu pinjaman sebelumnya dapat memiliki banyak baris.

credit_card_balance: data bulanan tentang kartu kredit sebelumnya yang dimiliki klien dengan Home Credit. Setiap baris adalah satu bulan dari saldo kartu kredit, dan satu kartu kredit dapat memiliki banyak baris.

installments_payment: riwayat pembayaran untuk pinjaman sebelumnya di Home Credit. Ada satu baris untuk setiap pembayaran yang dilakukan dan satu baris untuk setiap pembayaran yang terlewatkan.



![image](https://drive.google.com/uc?id=1uMUlwp_cnYv8sIfGPgiXf-uxBxAzptqM)


In [2]:
#import libraries
import pandas as pd
import numpy as np


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
X_train_selected_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X_train_hci.csv')
X_test_selected_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X_test_hci.csv')
y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y_train_hci.csv')
y_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y_test_hci.csv')
application_bureau_prev_test_selected_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/application_bureau_prev_test_selected_features.csv')

In [5]:
y_train = y_train['TARGET']
y_test = y_test['TARGET']

In [6]:
X_train_selected_features.drop(['Unnamed: 0.2'], axis='columns', inplace=True)
X_test_selected_features.drop(['Unnamed: 0.2'], axis='columns', inplace=True)
application_bureau_prev_test_selected_features.drop(['Unnamed: 0.2'], axis='columns', inplace=True)

In [7]:
application_bureau_prev_test_selected_features.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'SK_ID_CURR', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
       ...
       'CREDIT_AMT_PAYMENT_TOTAL_CURRENT', 'CREDIT_AMT_RECEIVABLE_PRINCIPAL',
       'CREDIT_AMT_RECIVABLE', 'CREDIT_AMT_TOTAL_RECEIVABLE',
       'CREDIT_CNT_DRAWINGS_ATM_CURRENT', 'CREDIT_CNT_DRAWINGS_CURRENT',
       'CREDIT_CNT_DRAWINGS_POS_CURRENT', 'CREDIT_CNT_INSTALMENT_MATURE_CUM',
       'CREDIT_SK_DPD', 'CREDIT_SK_DPD_DEF'],
      dtype='object', length=115)

In [8]:
y_train.info()

<class 'pandas.core.series.Series'>
RangeIndex: 246008 entries, 0 to 246007
Series name: TARGET
Non-Null Count   Dtype
--------------   -----
246008 non-null  int64
dtypes: int64(1)
memory usage: 1.9 MB


In [9]:
X_train_selected_features.shape

(246008, 115)

In [10]:
def check_data(X, y):
    # Check for missing values
    if X.isnull().sum().sum() > 0:
        print("Warning: Input data contains missing values. Imputing...")
        X.fillna(X.median(), inplace=True)
        
    # Check for invalid values
    # if X.isin([np.inf, -np.inf]).sum().sum() > 0:
    #     print("Warning: Input data contains invalid values. Removing...")
    #     X = X.replace([np.inf, -np.inf], np.nan).dropna()
    #     y = y.loc[X.index]
        
    # Check target variable is binary
    if set(y.unique()) != set([0, 1]):
        raise ValueError("Target variable is not binary")
        
    return X, y
    


In [11]:
# check_data(X_train_selected_features, y_train)

In [12]:
# check_data(X_test_selected_features, y_test)

# Imputasi Data sebelum menerapkan algoritma pembelajaran mesin
Ada banyak cara untuk menangani nilai yang hilang.

 Kita dapat menggunakan fillna() dan mengganti nilai yang hilang dengan rata-rata data, median, atau nilai paling sering. Pendekatan yang akan kita gunakan di bawah ini adalah Iterative Imputer. Iterative imputer akan menganggap variabel yang hilang sebagai variabel dependen dan semua fitur lainnya akan menjadi variabel independen. Kemudian akan menerapkan regresi dan variabel independen akan digunakan untuk menentukan variabel dependen (yang merupakan fitur yang hilang).

In [13]:
print(application_bureau_prev_test_selected_features.columns.get_loc("SK_ID_CURR")
)
print(X_train_selected_features.columns.get_loc("SK_ID_CURR")
)

2
2


In [14]:
X_train_selected_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246008 entries, 0 to 246007
Columns: 115 entries, Unnamed: 0.1 to CREDIT_SK_DPD_DEF
dtypes: float64(110), int64(5)
memory usage: 215.8 MB


In [15]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge, Ridge

# Create the imputer object
imputer = IterativeImputer(BayesianRidge())

# Impute missing values for X_train_selected_features
X_train_imputed = pd.DataFrame(index=X_train_selected_features.index)


for column in X_train_selected_features.columns:
    column_values = X_train_selected_features[column].values.reshape(-1, 1)
    imputed_values = imputer.fit_transform(column_values)
    X_train_imputed = pd.concat([X_train_imputed, pd.DataFrame(imputed_values, columns=[column])], axis=1)

# Impute missing values for application_bureau_prev_test_selected_features_subset1
application_bureau_prev_test_selected_features_subset1 = application_bureau_prev_test_selected_features.iloc[:, np.r_[113:115, 0:57]]
app_bur_prev_test_imputed_subset1 = pd.DataFrame(index=application_bureau_prev_test_selected_features_subset1.index)

for column in application_bureau_prev_test_selected_features_subset1.columns:
    column_values = application_bureau_prev_test_selected_features_subset1[column].values.reshape(-1, 1)
    imputed_values = imputer.fit_transform(column_values)
    app_bur_prev_test_imputed_subset1 = pd.concat([app_bur_prev_test_imputed_subset1, pd.DataFrame(imputed_values, columns=[column])], axis=1)

# Impute missing values for application_bureau_prev_test_selected_features_subset2
application_bureau_prev_test_selected_features_subset2 = application_bureau_prev_test_selected_features.iloc[:, np.r_[57:]]
app_bur_prev_test_imputed_subset2 = pd.DataFrame(index=application_bureau_prev_test_selected_features_subset2.index)

for column in application_bureau_prev_test_selected_features_subset2.columns:
    column_values = application_bureau_prev_test_selected_features_subset2[column].values.reshape(-1, 1)
    imputed_values = imputer.fit_transform(column_values)
    app_bur_prev_test_imputed_subset2 = pd.concat([app_bur_prev_test_imputed_subset2, pd.DataFrame(imputed_values, columns=[column])], axis=1)


# Merge the imputed subsets
app_bur_prev_test_imputed = pd.merge(app_bur_prev_test_imputed_subset1, app_bur_prev_test_imputed_subset2, on='SK_ID_CURR')

# Impute missing values for X_test_selected_features
X_test_imputed = pd.DataFrame(index=X_test_selected_features.index)

for column in X_test_selected_features.columns:
    column_values = X_test_selected_features[column].values.reshape(-1, 1)
    imputed_values = imputer.fit_transform(column_values)
    X_test_imputed = pd.concat([X_test_imputed, pd.DataFrame(imputed_values, columns=[column])], axis=1)

# Print the shapes of the imputed datasets
print(X_test_imputed.shape)
print(X_train_imputed.shape)
print(app_bur_prev_test_imputed.shape)


(61503, 115)
(246008, 115)
(48744, 115)


In [16]:
# Print the column names of app_bur_prev_test_imputed
print("Columns of app_bur_prev_test_imputed:")
print(app_bur_prev_test_imputed.columns)

# Print the column names of X_train_imputed
print("Columns of X_train_imputed:")
print(X_train_imputed.columns)

# Find the columns that are present in one DataFrame but not the other
missing_columns_in_app_bur_prev_test_imputed = set(X_train_imputed.columns) - set(app_bur_prev_test_imputed.columns)
missing_columns_in_X_train_imputed = set(app_bur_prev_test_imputed.columns) - set(X_train_imputed.columns)

print("Missing columns in app_bur_prev_test_imputed:")
print(missing_columns_in_app_bur_prev_test_imputed)

print("Missing columns in X_train_imputed:")
print(missing_columns_in_X_train_imputed)


Columns of app_bur_prev_test_imputed:
Index(['CREDIT_SK_DPD', 'CREDIT_SK_DPD_DEF', 'Unnamed: 0.1_x', 'Unnamed: 0_x',
       'SK_ID_CURR', 'AMT_INCOME_TOTAL_x', 'AMT_CREDIT_x', 'AMT_ANNUITY_x',
       'AMT_GOODS_PRICE_x', 'DAYS_BIRTH_x',
       ...
       'BUREAU_DAYS_CREDIT_ENDDATE_y', 'BUREAU_DAYS_ENDDATE_FACT_y',
       'BUREAU_AMT_CREDIT_MAX_OVERDUE_y', 'BUREAU_AMT_CREDIT_SUM_y',
       'BUREAU_AMT_CREDIT_SUM_DEBT_y', 'BUREAU_AMT_CREDIT_SUM_LIMIT_y',
       'BUREAU_AMT_CREDIT_SUM_OVERDUE_y', 'BUREAU_DAYS_CREDIT_UPDATE_y',
       'BUREAU_AMT_ANNUITY_y', 'BUREAU_CREDIT_ACTIVE_Active_y'],
      dtype='object', length=115)
Columns of X_train_imputed:
Index(['Unnamed: 0.1', 'Unnamed: 0', 'SK_ID_CURR', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
       ...
       'CREDIT_AMT_PAYMENT_TOTAL_CURRENT', 'CREDIT_AMT_RECEIVABLE_PRINCIPAL',
       'CREDIT_AMT_RECIVABLE', 'CREDIT_AMT_TOTAL_RECEIVABLE',
      

Sejajarkan kerangka data pelatihan dan pengujian, pertahankan hanya kolom yang ada di kedua kerangka data Kita lihat di atas bahwa jumlah kolom dalam test dan training set tidak sama.

In [17]:
app_bur_prev_test_imputed.columns

Index(['CREDIT_SK_DPD', 'CREDIT_SK_DPD_DEF', 'Unnamed: 0.1_x', 'Unnamed: 0_x',
       'SK_ID_CURR', 'AMT_INCOME_TOTAL_x', 'AMT_CREDIT_x', 'AMT_ANNUITY_x',
       'AMT_GOODS_PRICE_x', 'DAYS_BIRTH_x',
       ...
       'BUREAU_DAYS_CREDIT_ENDDATE_y', 'BUREAU_DAYS_ENDDATE_FACT_y',
       'BUREAU_AMT_CREDIT_MAX_OVERDUE_y', 'BUREAU_AMT_CREDIT_SUM_y',
       'BUREAU_AMT_CREDIT_SUM_DEBT_y', 'BUREAU_AMT_CREDIT_SUM_LIMIT_y',
       'BUREAU_AMT_CREDIT_SUM_OVERDUE_y', 'BUREAU_DAYS_CREDIT_UPDATE_y',
       'BUREAU_AMT_ANNUITY_y', 'BUREAU_CREDIT_ACTIVE_Active_y'],
      dtype='object', length=115)

In [18]:
# X_train_imputed, app_bur_prev_test_imputed = app_bur_prev_test_imputed.align(X_train_imputed, join='inner', axis=1)
# X_train_imputed, X_test_imputed = X_train_imputed.align(X_test_imputed, join='inner', axis=1)
# print(X_test_imputed.shape)
# print(X_train_imputed.shape)
# print(app_bur_prev_test_imputed.shape)

# Menerapkan Algoritma Pembelajaran Mesin (menggunakan validasi silang)


> Menerapkan Regresi Logistik

In [19]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(random_state = 0, class_weight='balanced', solver='lbfgs', max_iter=10000)
lr_clf.fit(X_train_imputed, y_train)



LogisticRegression(class_weight='balanced', max_iter=10000, random_state=0)

In [20]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
y_train_pred_lr=cross_val_predict(lr_clf, X_train_imputed, y_train, cv=10)
print('Akurasi pada set pelatihan:',cross_val_score(lr_clf, X_train_imputed,y_train, cv=10, scoring='accuracy'))
print('Akurasi pada set tes:',cross_val_score(lr_clf, X_test_imputed,y_test, cv=10, scoring='accuracy'))

Akurasi pada set pelatihan: [0.62603146 0.62196659 0.62241372 0.62391773 0.6198122  0.62826714
 0.62017804 0.62054388 0.62573171 0.61963415]
Akurasi pada set tes: [0.63810762 0.62347586 0.617298   0.63398374 0.62910569 0.63170732
 0.62130081 0.62406504 0.63089431 0.62861789]


> XGBoost

In [21]:
#Skala_pos_berat jika disetel ke jumlah(instance negatif)/ sum(instance negatif)
#akan menangani data yang tidak seimbang dalam kumpulan data
scale_pos_weight_value=y_train.value_counts().values.tolist()[0]/y_train.value_counts().values.tolist()[1]
from xgboost import XGBClassifier
XGB_clf = XGBClassifier(scale_pos_weight=scale_pos_weight_value)
XGB_clf.fit(X_train_imputed, y_train)
print('Akurasi pada set pelatihan:',cross_val_score(XGB_clf, X_train_imputed, y_train, cv=10, scoring='accuracy'))
print('Akurasi pada set tes:',cross_val_score(XGB_clf, X_test_imputed, y_test, cv=10, scoring='accuracy'))

Akurasi pada set pelatihan: [0.75102638 0.75350596 0.74935978 0.75570099 0.75667656 0.75480672
 0.75643267 0.74692086 0.75341463 0.75097561]
Akurasi pada set tes: [0.82295562 0.81076248 0.81287596 0.81252033 0.80943089 0.81317073
 0.80650407 0.81837398 0.8095935  0.81365854]


> CATBOOST

In [28]:
cols_numeric = X_train_imputed.select_dtypes([np.number]).columns 
cols_categorical=X_train_imputed.columns.difference(cols_numeric)
# print(y_train.dtype)
# print(np.unique(y_train))
# print(y_test.dtype)
# print(np.unique(y_test))

In [41]:
cols_numeric

Index(['Unnamed: 0.1', 'Unnamed: 0', 'SK_ID_CURR', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
       ...
       'CREDIT_AMT_PAYMENT_TOTAL_CURRENT', 'CREDIT_AMT_RECEIVABLE_PRINCIPAL',
       'CREDIT_AMT_RECIVABLE', 'CREDIT_AMT_TOTAL_RECEIVABLE',
       'CREDIT_CNT_DRAWINGS_ATM_CURRENT', 'CREDIT_CNT_DRAWINGS_CURRENT',
       'CREDIT_CNT_DRAWINGS_POS_CURRENT', 'CREDIT_CNT_INSTALMENT_MATURE_CUM',
       'CREDIT_SK_DPD', 'CREDIT_SK_DPD_DEF'],
      dtype='object', length=115)

In [24]:
# pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.3 MB/s eta 0:00:00


In [43]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

CatBoost_reg = CatBoostRegressor(iterations=50, depth=3, learning_rate=0.1, loss_function='RMSE')
CatBoost_reg.fit(X_train_imputed, y_train) 

# Evaluate performance on training set
y_train_pred = CatBoost_reg.predict(X_train_imputed)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
print('Root Mean Squared Error on training set:', train_rmse)

cv_scores = cross_val_score(CatBoost_reg, X_train_imputed, y_train, cv=10, scoring='neg_root_mean_squared_error')
cv_rmse_mean = -cv_scores.mean()
print('Cross-validated RMSE on training set:', cv_rmse_mean)

# Evaluate performance on test set
y_test_pred = CatBoost_reg.predict(X_test_imputed)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print('Root Mean Squared Error on test set:', test_rmse)


0:	learn: 0.2713396	total: 62.9ms	remaining: 3.08s
1:	learn: 0.2704779	total: 118ms	remaining: 2.83s
2:	learn: 0.2697623	total: 185ms	remaining: 2.9s
3:	learn: 0.2691397	total: 248ms	remaining: 2.85s
4:	learn: 0.2685700	total: 313ms	remaining: 2.82s
5:	learn: 0.2680761	total: 402ms	remaining: 2.94s
6:	learn: 0.2676318	total: 496ms	remaining: 3.04s
7:	learn: 0.2672334	total: 618ms	remaining: 3.24s
8:	learn: 0.2669017	total: 726ms	remaining: 3.31s
9:	learn: 0.2666061	total: 868ms	remaining: 3.47s
10:	learn: 0.2663378	total: 971ms	remaining: 3.44s
11:	learn: 0.2660846	total: 1.07s	remaining: 3.39s
12:	learn: 0.2658616	total: 1.19s	remaining: 3.38s
13:	learn: 0.2656495	total: 1.29s	remaining: 3.32s
14:	learn: 0.2654555	total: 1.43s	remaining: 3.35s
15:	learn: 0.2652944	total: 1.54s	remaining: 3.28s
16:	learn: 0.2651368	total: 1.65s	remaining: 3.21s
17:	learn: 0.2649866	total: 1.79s	remaining: 3.19s
18:	learn: 0.2648457	total: 1.9s	remaining: 3.11s
19:	learn: 0.2647098	total: 2.03s	remainin

Nilai RMSE yang rendah menunjukkan bahwa model membuat prediksi yang akurat baik pada set pelatihan maupun pengujian. Namun, penting untuk mempertimbangkan skala variabel target Anda untuk menginterpretasikan nilai absolut

Jika variabel target berada pada skala yang berbeda atau memiliki rentang nilai yang besar, akan sangat membantu jika membandingkan RMSE dengan rentang variabel target untuk lebih memahami kinerja model.

Secara keseluruhan, memiliki nilai RMSE yang serupa untuk set pelatihan dan pengujian dan pencapaian nilai RMSE yang rendah menunjukkan bahwa model CatBoostRegressor bekerja dengan baik pada tugas regresi ini.

#LightGBM

In [45]:
import re
import lightgbm as lgb

# Preprocess feature names
def preprocess_feature_names(names):
    # Remove special characters
    names = [re.sub(r'\W+', '', name) for name in names]
    # Replace spaces with underscores
    names = [name.replace(' ', '_') for name in names]
    # Ensure valid JSON strings
    names = [name.encode('utf-8').decode('unicode_escape').replace('"', '') for name in names]
    return names

# Preprocess feature names for training data
X_train_imputed.columns = preprocess_feature_names(X_train_imputed.columns)

# Create and fit LightGBM classifier
LightGBM_clf = lgb.LGBMClassifier(scale_pos_weight=scale_pos_weight_value)
LightGBM_clf.fit(X_train_imputed, y_train)

# Preprocess feature names for test data
X_test_imputed.columns = preprocess_feature_names(X_test_imputed.columns)

# Evaluate model performance
print('Accuracy on training set:', cross_val_score(LightGBM_clf, X_train_imputed, y_train, cv=10, scoring='accuracy'))
print('Accuracy on test set:', cross_val_score(LightGBM_clf, X_test_imputed, y_test, cv=10, scoring='accuracy'))

Accuracy on training set: [0.71391407 0.71427991 0.71180033 0.71204423 0.71476769 0.71623105
 0.71013373 0.71269461 0.71569106 0.71386179]
Accuracy on test set: [0.75727524 0.7483336  0.74280605 0.75154472 0.74731707 0.75203252
 0.7396748  0.75414634 0.75252033 0.75414634]


>RandomForest

In [46]:
#class_weight = 'balanced' ensures that RandomForest works well on imbalanced datasets.
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators = 10, random_state = 0, n_jobs=-1, class_weight="balanced")
rf_clf.fit(X_train_imputed, y_train)
print('Akurasi pada set pelatihan:',cross_val_score(rf_clf, X_train_imputed, y_train, cv=10, scoring='accuracy'))
print('Akurasi pada set tes:',cross_val_score(rf_clf, X_test_imputed, y_test, cv=10, scoring='accuracy'))

Akurasi pada set pelatihan: [0.91849925 0.91866184 0.91910898 0.91858055 0.91914963 0.91752368
 0.91870249 0.9185399  0.91768293 0.91865854]
Akurasi pada set tes: [0.9187124  0.91773695 0.91773695 0.91772358 0.91804878 0.91739837
 0.91853659 0.91886179 0.91837398 0.91902439]
